In [1]:
%reset
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
from torch.optim import lr_scheduler
import copy
import time
from collections import defaultdict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


cuda:0


Dataloader

In [2]:
# We read the dataset and create an iterable.
class features_from_views(data.Dataset):
    def __init__(self, features_path, gt_path):
        
        self.features = features_path
        self.ground_truth = gt_path
        
        
    def __getitem__(self, index):   # Function that returns one point and one label.
        self.feat = torch.load(self.features[index])
        self.feat = self.feat.permute(1, 0, 2)
        self.feat = torch.flatten(self.feat, start_dim = 1)
        self.gt = torch.load(self.ground_truth[index])
        self.gt = torch.sparse.FloatTensor(self.gt[0].unsqueeze(0), self.gt[1], 
                              size = torch.Size([self.feat.shape[0]])).to_dense()
        return  self.feat, self.gt
    
    def __len__(self):   # Length of the dataset.
        return len(self.features)

In [3]:
# We create the dataloader.
path_features = 'LEARNING/features/*.pt'
path_ground_truth = 'LEARNING/ground_truth_3D/*.pt'
list_features = glob.glob(path_features)
list_ground_truth = glob.glob(path_ground_truth)
train_features = features_from_views(list_features, list_ground_truth)
batch_size = 1
feature_loader = data.DataLoader(train_features, batch_size=batch_size, num_workers=0)

In [4]:
#for feat, gt in feature_loader:
#    print(feat.shape, gt.shape)

In [5]:
# We build a simple model with the inputs and one output layer.
class my_model(nn.Module):
    def __init__(self,n_in = 504,n_hidden=10,n_out= 6 ):
        super(my_model,self).__init__()
        self.n_in  = n_in
        self.n_out = n_out
         
        self.linearlinear = nn.Sequential(
            nn.Linear(self.n_in,self.n_out,bias=True),   # Hidden layer.
            )
       # self.logprob = nn.LogSoftmax(dim=1)                 # -Log(Softmax probability).
    
    def forward(self,x):
        x = self.linearlinear(x)
       # x = self.logprob(x)
        return x


In [6]:
# Now, we create the mode, the loss function or criterium and the optimizer 
# that we are going to use to minimize the loss.

# Model.
model = my_model().to(device)

# Negative log likelihood loss.
#criterium = nn.NLLLoss()
criterium = nn.CrossEntropyLoss()
#criterium = my_loss()
# Adam optimizer with learning rate 0.1 and L2 regularization with weight 1e-4.
optimizer = torch.optim.Adam(model.parameters(),lr=0.1,weight_decay=1e-4)

In [7]:
ind = np.linspace(-25, 0, 50)
2**ind

array([2.98023224e-08, 4.24460070e-08, 6.04537956e-08, 8.61014184e-08,
       1.22630088e-07, 1.74656105e-07, 2.48754246e-07, 3.54288647e-07,
       5.04596191e-07, 7.18671960e-07, 1.02356973e-06, 1.45782088e-06,
       2.07630380e-06, 2.95717912e-06, 4.21176725e-06, 5.99861647e-06,
       8.54353943e-06, 1.21681502e-05, 1.73305080e-05, 2.46830047e-05,
       3.51548102e-05, 5.00692966e-05, 7.13112785e-05, 1.01565206e-04,
       1.44654413e-04, 2.06024287e-04, 2.93430432e-04, 4.17918778e-04,
       5.95221511e-04, 8.47745221e-04, 1.20740253e-03, 1.71964506e-03,
       2.44920731e-03, 3.48828757e-03, 4.96819934e-03, 7.07596613e-03,
       1.00779565e-02, 1.43535462e-02, 2.04430619e-02, 2.91160647e-02,
       4.14686033e-02, 5.90617268e-02, 8.41187620e-02, 1.19806286e-01,
       1.70634301e-01, 2.43026185e-01, 3.46130446e-01, 4.92976861e-01,
       7.02123109e-01, 1.00000000e+00])

# Taining.
for k, (data, target) in enumerate(feature_loader):
    # Definition of inputs as variables for the net.
    # requires_grad is set False because we do not need to compute the 
    # derivative of the inputs.
    data   = Variable(data,requires_grad=False)
    target = Variable(target.long(),requires_grad=False)
    print(data.shape, target.shape)
    # Set gradient to 0.
    optimizer.zero_grad()
    # Feed forward.
    pred = model(data)
    pred = pred.permute(0, 2, 1)
    # Loss calculation.
    print(pred.max())
    loss = criterium(pred,target)
    # Gradient calculation.
    loss.backward()
    
    # Print loss every 10 iterations.
    if k%10==0:
        print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
        
    # Model weight modification based on the optimizer. 
    optimizer.step()

In [8]:
def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    train_loss = []
    val_loss = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            i = -25.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.param_groups[0]['lr'] = 2**i
                i = i + 1
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = outputs.permute(0, 2, 1)
                    #loss = calc_loss(outputs, labels, metrics)
                    loss = criterium(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        train_loss.append(loss.detach().cpu().numpy())
                    else: 
                        val_loss.append(loss.detach().cpu().numpy())
                    print(loss, optimizer.param_groups[0]['lr'])
                # statistics
                epoch_samples += inputs.size(0)
            print(loss)
            #print_metrics(metrics, epoch_samples, phase)
            #epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            #if phase == 'val' and epoch_loss < best_loss:
            #    print("saving best model")
            #    best_loss = epoch_loss
            #    best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(loss))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model, val_loss, train_loss

In [ ]:
batch_size = 4

dataloaders = {
    'train':feature_loader,
    'val': feature_loader
    }
num_class = 6

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
print(device)
model = train_model(model, optimizer, exp_lr_scheduler, num_epochs=1)

cuda:0
Epoch 0/0
----------
LR 0.1
tensor(2.0765, device='cuda:0', grad_fn=<NllLoss2DBackward>) 2.9802322387695312e-08
tensor(2.0792, device='cuda:0', grad_fn=<NllLoss2DBackward>) 5.960464477539063e-08
tensor(2.0755, device='cuda:0', grad_fn=<NllLoss2DBackward>) 1.1920928955078125e-07
tensor(2.0783, device='cuda:0', grad_fn=<NllLoss2DBackward>) 2.384185791015625e-07
tensor(2.0727, device='cuda:0', grad_fn=<NllLoss2DBackward>) 4.76837158203125e-07
tensor(2.0848, device='cuda:0', grad_fn=<NllLoss2DBackward>) 9.5367431640625e-07
tensor(2.0786, device='cuda:0', grad_fn=<NllLoss2DBackward>) 1.9073486328125e-06
tensor(2.0763, device='cuda:0', grad_fn=<NllLoss2DBackward>) 3.814697265625e-06
tensor(2.0840, device='cuda:0', grad_fn=<NllLoss2DBackward>) 7.62939453125e-06
tensor(2.0735, device='cuda:0', grad_fn=<NllLoss2DBackward>) 1.52587890625e-05
tensor(2.0810, device='cuda:0', grad_fn=<NllLoss2DBackward>) 3.0517578125e-05
tensor(2.0634, device='cuda:0', grad_fn=<NllLoss2DBackward>) 6.10351562

In [9]:
train_loss = model[1]
val_loss = model[2]
plt.plot(train_loss, label = 'train')
plt.plot(val_loss, label = 'val')
plt.xlabel('Batch number')
plt.ylabel('Epoch number')
plt.show()

NameError: name 'pred' is not defined

In [ ]:
torch.unique(pred_arg, return_counts = True)

In [10]:
for inputs, labels in dataloaders["train"]:
    inputs = inputs.to(device)
    outputs = model(inputs)
    break

In [11]:
arg = torch.argmax(outputs, dim = 2)

In [12]:
torch.unique(arg)

tensor([0, 2], device='cuda:0')